# Example Run with PBC SchNet
I'm just going to take one of our best-trained models and run it on a moderate supercell

In [1]:
from fff.learning.gc.ase import SchnetCalculator
from fff.learning.gc.models import load_pretrained_model
from ase.build import molecule
import numpy as np
import torch

/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuration

In [2]:
model_path = '../../colmena/initial/notebooks/prod-runs/ttm_md-25000_thr-0.001_T-300/final-model/model'  # Model and weights

## Make the calculator
Load the model and give it to the ASE wrapper

In [3]:
model = torch.load(model_path, map_location='cpu')

In [4]:
calc = SchnetCalculator(model, device='cpu')  # I don't think we support CUDA yet

## Run a supercell computation
You _must_ use a large cell. Our code assumes a cell larger than the cutoff radius for SchNet (and I haven't put in any warnings about it yet)

In [5]:
water = molecule('H2O')

Make the water in the center of a periodic cell

In [6]:
water.cell = [5.] * 3
water.pbc = True
water

Atoms(symbols='OH2', pbc=True, cell=[5.0, 5.0, 5.0])

Duplicate it big (512 waters)

In [7]:
water *= [8, 8, 8]
water

Atoms(symbols='H1024O512', pbc=True, cell=[40.0, 40.0, 40.0])

See how long it takes to get forces

In [8]:
%%time
forces = calc.get_forces(water)

CPU times: user 2.41 s, sys: 549 ms, total: 2.96 s
Wall time: 256 ms


Make sure the forces are identical for each water

In [9]:
max_diff = 0
for start in range(0, len(water), 3):
    max_diff = max(max_diff, np.abs(forces[:3, :] - forces[start:start+3, :]).max())
    assert np.isclose(forces[:3, :], forces[start:start+3, :], atol=1e-2).all()
print(f'Maximum disagreement between forces: {max_diff:.2e} eV/A')

Maximum disagreement between forces: 4.67e-03 eV/A
